<a href="https://colab.research.google.com/github/GVCHEMPION/Sirius_AI/blob/main/Parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Задание на второй этап отбора**

*Создать новые признаки на основе имеющихся данных*

***для столбца Жалобы***

## **Установка доп. библиотек**

In [1]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 50.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=78ba046a51b626560b7c32f24230f72e79bee7a5996ed660909397489db4cb5a
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [2]:
!pip install -U pymorphy2-dicts-ru

In [3]:
import numpy as np
import pandas as pd
import gdown

from itertools import groupby #Библиотека для приведения слов к начальной форме

## **Загрузка датасета**

In [5]:
df = pd.read_csv('med_prot.csv', sep=';', header=0, index_col=0) # Загружаем базу
df

,MedicalRecordDate,PatientKey,Возраст,Пол,СпециальностьВрача,Жалобы,ПеренесенныеЗаболевания,ПеренесенныеОперации,ПринимаемыеПрепараты
MedicalRecordKey,,,,,,,,,
106627476,2023-09-01 18:39:46.000,0xA52FAC0660E364A3BD2338AD6749D320FF3E08338230...,9764,1.0,Врач акушер гинеколог,на мажущие выделения из половых путей,"Факт получения травм, ожогов, ран, проведения ...",Отрицает,Левотироксин 75 мкг/сут
106627477,2023-09-01 18:39:45.000,0xC03852BF5FDC992B86A87B1215BC47EBF66C1AB03F6D...,13814,1.0,Врач уролог,на дискомфорт при мочеиспускании на учащенное ...,"Факт получения травм, ожогов, ран, проведения ...",аппендэктомия 2013. кесарево сечение - пре...,"монурал, канефрон."
106627482,2023-09-01 18:39:53.000,0x417FA75C8DAD480F952C6931B305A0218311A4BE61E2...,10311,0.0,Врач дерматовенеролог,На покраснение и шелушение кожи головы.,ГЭРБ ЖДА,-,Колекальльциферол - 4000 МЕ Калия йодид Мальто...
106627486,2023-09-01 18:39:57.000,0x922886B7EA4058AC1BA576BEC37C24A4A2E8AB65E912...,22669,1.0,Врач кардиолог,Подписано информированное добровольное согласи...,узловой зоб,"аппендицит в 15-летнем возрасте, удаление кист...","на пенициллиновый ряд - отек квинке, сульфанил..."
106627490,2023-09-01 18:40:03.000,0xFAB438E0CA9FFC5CFF28E0B33914E95FEE9524108639...,15586,1.0,Врач акушер гинеколог,Жалобы на болезненность и нагрубание правой мо...,"Факт получения травм, ожогов, ран, проведения ...",аппендэктомия в 5 лет Импланты молочных желе...,Не отягощен
...,...,...,...,...,...,...,...,...,...
106627461,2023-09-01 18:39:19.000,0x5CD4B5F0F734ABAD1F5DE02CDBFC06C095E53C53F0FC...,17642,0.0,Врач терапевт,покраснение отек 1 пальца правой стопы,"Факт получения травм, ожогов, ран, проведения ...",варикоцеле,На постоянной основе никакие лекарственные пре...
106627464,2023-09-01 18:39:25.000,0x2D092FE67C4C4A48BB11D7105FBF273125834A0CCACE...,13334,1.0,Врач оториноларинголог,на момент осмотра активных жалоб по ЛОР не пре...,"Факт получения травм, ожогов, ран, проведения ...",Отрицает,Неизвестен
106627465,2023-09-01 18:39:24.000,0x3FEEB9EE504FE5396E8A7648DA7332A96AB15EB92F11...,8429,0.0,Врач уролог,НАЛ жалобы на возникновение болевых ощущенив ...,"ОРВИ Факт получения травм, ожогов, ран, провед...",нет,NaN


## **Будем парсить колонку с ЖАЛОБАМИ**

In [6]:
#Удалим ненужные столбцы
df = df.drop('MedicalRecordDate', axis=1)
df = df.drop('PatientKey', axis=1)
df = df.drop('ПеренесенныеЗаболевания', axis=1)
df = df.drop('ПеренесенныеОперации', axis=1)
df = df.drop('ПринимаемыеПрепараты', axis=1)

In [7]:
#удаление строк с отрицанием жалоб
search_list = ["отрицает", "не предъя", "плановый осмотр" ,"нет", "жалоб нет", "не имеет"]
score = 0
del_list = []
for index in range(0 , df['Жалобы'].shape[0]):
  for i in search_list:
    if i in df['Жалобы'].values[index].lower():
        df['Жалобы'].values[index]=np.nan
        break

In [8]:
df

,Возраст,Пол,СпециальностьВрача,Жалобы
MedicalRecordKey,,,,
106627476,9764,1.0,Врач акушер гинеколог,на мажущие выделения из половых путей
106627477,13814,1.0,Врач уролог,на дискомфорт при мочеиспускании на учащенное ...
106627482,10311,0.0,Врач дерматовенеролог,На покраснение и шелушение кожи головы.
106627486,22669,1.0,Врач кардиолог,Подписано информированное добровольное согласи...
106627490,15586,1.0,Врач акушер гинеколог,Жалобы на болезненность и нагрубание правой мо...
...,...,...,...,...
106627461,17642,0.0,Врач терапевт,покраснение отек 1 пальца правой стопы
106627464,13334,1.0,Врач оториноларинголог,NaN
106627465,8429,0.0,Врач уролог,НАЛ жалобы на возникновение болевых ощущенив ...


In [9]:
#Сколько в итоге удалено незначащих жалоб
df.isnull().sum()

Возраст                  0
Пол                      1
СпециальностьВрача       0
Жалобы                2948
dtype: int64

In [10]:
# Удалим строки с пустыми значениями
df = df.dropna()

In [11]:
complaints=list(df['Жалобы'])
index_complaints=[] #Список со списками жалоб (Длинна списка соответствует количеству записей в df)
all_complaints=[] #Список со всеми жалобами

for comp in complaints[:]:
  #========= Раздел фильтрации=====
  temp = comp.lower()
  while '  ' in temp:
    temp = temp.replace('  ',' ')

  temp = temp.replace('жалоб ','')
  temp = temp.replace('на ','')
  temp = temp.replace('.',',')
  temp = temp.replace(';',',')
  temp = temp.replace(', ',',')
  temp = temp.strip()
  #=================================
  temp = temp.split(',')
  if temp[-1]=='':
    temp=temp[:-1]
  index_complaints.append(temp) #Список со списками жалоб (Длинна списка соответствует количеству записей в df)
  all_complaints.extend(temp) #Список со всеми жалобами

In [12]:
n=35
print(f'Количество записей в таблице df - {len(df)}')
print(f'Количество записей в index_complaints - {len(index_complaints)}')
print(f'Количество записей в all_complaints - {len(all_complaints)}')
print(f'Пример {n}-ой записи index_complaints\n{index_complaints[n]}')
print(f'Пример {n}-ой записи all_complaints\n{all_complaints[n]}')

Количество записей в таблице df - 9816
Количество записей в index_complaints - 9816
Количество записей в all_complaints - 31518
Пример 35-ой записи index_complaints
['першение в горле ', 'кашель со скудной мокротой', 'жар во лбу']
Пример 35-ой записи all_complaints
зеленые выделения из носа


## **Преобразование слов к начальной форме**

In [13]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [14]:
def get_normal_form(lst): #Функция перевода значений в списке к нормальным
    ls=[]
    for line in lst:
      words=[]
      for word in line.split(' '):
            p = morph.parse(word)[0]
            words.append(p.normal_form)
      ls.append(' '.join(words))
    return ls

In [15]:
#Проверка работы
get_normal_form(['Варкалось', 'Хливкие шорьки', 'Пырялись по наве', 'И хрюкотали зелюки', 'Как мюмзики в мове', 'жалоб нет'])

['варкалось',
 'хливкий шорёк',
 'пыряться по нава',
 'и хрюкоталь зелюк',
 'как мюмзика в мов',
 'жалоба нет']

In [16]:
#Перевод всех жалоб к начальной   форме (Выполняется долго)
all_complaints=get_normal_form(all_complaints)

## **Создание словаря частотности жалоб**

In [17]:
#Создание словаря частотноти жалоб {'жалоба':количество} с уловием длинны жалобы > 2
slov=({key:len(list(group)) for key, group in groupby(sorted(all_complaints)) if len(key)>2})

In [18]:
#Сортировка словаря жалоб по значению (количеству жалоб)
slov = sorted(slov.items(), key=lambda x: x[1], reverse=True)

In [19]:
#Выведем первые 30 жалоб
for i, comp in enumerate(slov[:30]):
  print(i+1, comp)

1 ('слабость', 440)
2 ('общий слабость', 340)
3 ('заложенность нос', 278)
4 ('головной боль', 209)
5 ('утомляемость', 170)
6 ('насморк', 159)
7 ('дискомфорт', 153)
8 ('головокружение', 151)
9 ('боль в горло', 148)
10 ('першение в горло', 145)
11 ('повышенный утомляемость', 113)
12 ('тошнота', 106)
13 ('сонливость', 102)
14 ('вздутие живот', 93)
15 ('боль', 91)
16 ('кашель', 85)
17 ('метеоризм', 73)
18 ('быстрый утомляемость', 71)
19 ('учащённый мочеиспускание', 69)
20 ('изжога', 68)
21 ('потливость', 67)
22 ('осиплость голос', 66)
23 ('стекание слизь по задний стенка глотка', 65)
24 ('дискомфорт в горло', 64)
25 ('зуд', 63)
26 ('сухость в рот', 63)
27 ('тревожность', 62)
28 ('сухость кожа', 56)
29 ('без патологический примесь', 54)
30 ('выпадение волос', 49)


In [20]:
#Отберем N самых частых жалоб
n=15
comp_N=[comp[0] for comp in slov[:n]]
comp_N

['слабость',
 'общий слабость',
 'заложенность нос',
 'головной боль',
 'утомляемость',
 'насморк',
 'дискомфорт',
 'головокружение',
 'боль в горло',
 'першение в горло',
 'повышенный утомляемость',
 'тошнота',
 'сонливость',
 'вздутие живот',
 'боль']

In [21]:
#Создание массива жалоб для каждой записи df
add_inf=[]
for comp in get_normal_form(complaints): #Неревод к начальной форме всей базы Жалоб
  temp=[0]*len(comp_N)
  for i, c in enumerate(comp_N):
      if c in comp:
        temp[i]=1
  add_inf.append(temp)

add_inf_df=np.asarray(add_inf)

## **Посмотрим результат:**

In [22]:
#Добавление столбцов с новыми бинарными признаками
for i in range (len(comp_N)):
  df.insert(i+4, comp_N[i], add_inf_df[:,i], True)

In [23]:
df[100:150]

,Возраст,Пол,СпециальностьВрача,Жалобы,слабость,общий слабость,заложенность нос,головной боль,утомляемость,насморк,дискомфорт,головокружение,боль в горло,першение в горло,повышенный утомляемость,тошнота,сонливость,вздутие живот,боль
MedicalRecordKey,,,,,,,,,,,,,,,,,,,
106627886,16923,1.0,Врач физиотерапевт,"На боли в пояснице, усиливающиеся при длительн...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
106627887,9442,1.0,Врач акушер гинеколог,жалобы на дискомфорт во влагалище,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
106627889,13325,1.0,Врач терапевт,"на слабость, сонливость",1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
106627890,12501,1.0,Врач невролог,на боли в поясничном отделе позвоночника.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
106627891,10645,1.0,Врач уролог,"Эпизоды учащенного мочеиспускания, дискомфорт ...",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
106627904,15133,1.0,Врач терапевт,боли внизу живота и пояснице ВАШ 5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
106627907,10648,0.0,Врач гастроэнтеролог,"Жалобы на периодические изжогу, боли в эпигас...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
106627908,15799,0.0,Врач травматолог ортопед,"На боль в области правого коленного сустава , ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
106627910,12392,0.0,Врач оториноларинголог,Насморк: заложенность носа( чаще на улице )...,1,0,1,0,0,1,1,0,0,1,0,0,0,0,0


In [24]:
df.to_csv('output.csv')